In [ ]:
!pip install bitsandbytes accelerate transformers -qq -U

In [ ]:
!git clone https://github.com/werserk/HSE-AI-Assistant-Hack-Python-baseline/

In [ ]:
import sys
sys.path.append('/content/HSE-AI-Assistant-Hack-Python-baseline')

In [ ]:
!nvidia-smi

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = 'Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    "Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24",
    device_map='cuda',
    torch_dtype=torch.float16
)

In [ ]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"}
]

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe(messages)

In [ ]:
pipe(messages, max_new_tokens=100)

## Data analysis (train)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install openpyxl -q

In [ ]:
path = "/home/jovyan/"

In [ ]:
train_solutions = pd.read_excel(path + 'train/solutions.xlsx')
train_solutions = train_solutions.reset_index(drop=True)

In [ ]:
train_tasks = pd.read_excel(path + 'train/tasks.xlsx')
train_tasks = train_tasks.reset_index(drop=True)

In [ ]:
train_tests = pd.read_excel(path + 'train/tests.xlsx')
train_tests = train_tests.reset_index(drop=True)

In [ ]:
print(train_solutions.shape, train_tasks.shape, train_tests.shape)

In [ ]:
solution1 = train_solutions.loc[(train_solutions['task_id'] == 39) & (train_solutions['id'] == 619), 'student_solution'].item()
solution1

In [ ]:
task1 = train_tasks[train_tasks['id']==39]['description'].iloc[0]
task1

In [ ]:
comment = train_solutions[(train_solutions['task_id']==39) & (train_solutions['id'] == 619)]['author_comment'].item()
comment

### WORKING ZONE, DANILA ZDES

In [ ]:
system_prompt = """Ты преподаватель Высшей школы экономики. Твоя задача, получив ЗАДАЧУ и РЕШЕНИЕ дать подсказку ученику, где находится ошибка в его РЕШЕНИИ. Прямо напиши где ошибка.

Не пиши большой ответ. Пиши только подсказку, которая поможет студенту двигаться в правильном направлении, но не видеть весь ответ сразу. Люди не должны догадаться, что ты робот.

Пример:
Ввод:
ЗАДАЧА:
Реализуйте следующую программу:\n\nВ переменную agencies считывается список названий агентств. Этот код уже написан.\nПрограмма проверяет, какие названия агентств оканчиваются на "ум". Если такие строки есть, то программа сохраняет их в отдельный список.\nВ конце программа печатает отсортированный в алфавитном порядке список с названиями агентств, которые были туда сохранены.\nЕсли таких строк нет, программа печатает пустой список.
РЕШЕНИЕ СТУДЕНТА (НЕ СМОТРИ НА СПЕЦИАЛЬНЫЕ СИМВОЛЫ):
agencies = input().split()\nus = []\n\nfor agency in agencies:\n    if agency.endswith(ум):\n        us.append(agency)\n        \nprint(sorted(us))
ОЖИДАЕМЫЙ ОТВЕТ:
Вы забыли заключить "ум" в кавычки.
"""
prompt = f"""ЗАДАЧА: {task1}, РЕШЕНИЕ СТУДЕНТА: {solution1}"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]
answer = pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']
print(answer)

In [ ]:
# соберем наш датасет
final_train_df = pd.read_csv(path + 'train_database.csv')
train_solutions_df = pd.read_excel(path + 'train/solutions.xlsx')

In [ ]:
# по train_solutions_df идем и подтягиваем по task_id данные из final_train_df
description = 'a'
input = 'a'
output = 'a'
student_solution = 'a'
author_solution = 'a'
author_comment = 'a'
test_type = 'a'

res = []
for i in range(len(input)): # input из колонки input в new_df
  res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {test_type} \n<Ввод>: {input} \n<Вывод>: {output}")

result = ' '.join(res)
prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n<Комментарий эксперта>: {author_comment}"
print(prompt)

In [ ]:
final_train_df.shape, train_solutions_df.shape

In [ ]:
final_train_df.nunique()

In [ ]:
final_train_df.sample(2)

In [ ]:
train_solutions_df.sample(2)

In [ ]:
train_solutions_df.nunique()

In [ ]:
new_df = {'id': [], 'task_id': [], 'description': [],
          'input': [], 'output': [], 'student_solution': [],
          'author_solution': [], 'author_comment': [], 'type': []}

In [ ]:
for index, row in train_solutions_df.iterrows():
    new_df['id'].append(row['id'])
    new_df['task_id'].append(row['task_id'])
    new_df['student_solution'].append(row['student_solution'])
    new_df['author_comment'].append(row['author_comment'])
    inp = final_train_df[final_train_df['task_id'] == row['task_id']]['input'].tolist()
    out = final_train_df[final_train_df['task_id'] == row['task_id']]['output'].tolist()
    new_df['input'].append(inp)
    new_df['output'].append(out)
    test_type = final_train_df[final_train_df['task_id'] == row['task_id']]['type'].tolist()
    new_df['type'].append(test_type)
    description = final_train_df[final_train_df['task_id'] == row['task_id']]['description'].tolist()[0]
    new_df['description'].append(description)
    author_solution = final_train_df[final_train_df['task_id'] == row['task_id']]['author_solution'].tolist()[0]
    new_df['author_solution'].append(author_solution)

In [ ]:
new_df = pd.DataFrame(new_df)

In [ ]:
new_df.shape

In [ ]:
new_df.sample(3)

In [ ]:
final_prompt = []
for index, row in new_df.iterrows():
  description = row['description']
  res = []
  for i in range(len(row['input'])):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row['type'][i]} \n<Ввод>: {row['input'][i]} \n<Вывод>: {row['output'][i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: {author_comment}"
  final_prompt.append(prompt)

In [ ]:
new_df['prompt'] = final_prompt

In [ ]:
new_df.shape

In [ ]:
new_df.sample(2)

In [ ]:
new_df.to_csv('new_df_for_model.csv')

### Инференс

In [ ]:
path = "/home/jovyan/"

In [ ]:
final_test_df = pd.read_csv(path + 'test_database.csv')
test_solutions = pd.read_excel(path + 'test/solutions.xlsx')

In [ ]:
final_test_df.shape, test_solutions.shape

In [ ]:
test_solutions.sample(2)

In [ ]:
final_test_df.sample(2)

In [ ]:
new_test_df = {'id': [], 'task_id': [], 'description': [],
          'input': [], 'output': [], 'student_solution': [],
          'author_solution': [], 'author_comment': [], 'type': []}

In [ ]:
for index, row in test_solutions.iterrows():
    new_test_df['id'].append(row['id'])
    new_test_df['task_id'].append(row['task_id'])
    new_test_df['student_solution'].append(row['student_solution'])
    new_test_df['author_comment'].append('')
    inp = final_test_df[final_test_df['task_id'] == row['task_id']]['input'].tolist()
    out = final_test_df[final_test_df['task_id'] == row['task_id']]['output'].tolist()
    new_test_df['input'].append(inp)
    new_test_df['output'].append(out)
    test_type = final_test_df[final_test_df['task_id'] == row['task_id']]['type'].tolist()
    new_test_df['type'].append(test_type)
    description = final_test_df[final_test_df['task_id'] == row['task_id']]['description'].tolist()[0]
    new_test_df['description'].append(description)
    author_solution = final_test_df[final_test_df['task_id'] == row['task_id']]['author_solution'].tolist()[0]
    new_test_df['author_solution'].append(author_solution)

In [ ]:
new_test_df = pd.DataFrame(new_test_df)

In [ ]:
new_test_df.shape

In [ ]:
new_test_df.sample(2)

In [ ]:
final_test_prompt = []
for index, row in new_test_df.iterrows():
  description = row['description']
  res = []
  for i in range(len(row['input'])):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row['type'][i]} \n<Ввод>: {row['input'][i]} \n<Вывод>: {row['output'][i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: {author_comment}"
  final_test_prompt.append(prompt)

In [ ]:
new_test_df['prompt'] = final_test_prompt

In [ ]:
new_test_df.shape

In [ ]:
new_test_df.sample(2)

In [ ]:
print(new_test_df['prompt'][0])

In [ ]:
new_test_df.to_csv('new_test_df_for_inference.csv')

In [ ]:
system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. Write directly where the mistake is. Do not write a long answer. Provide only a hint that guides the student in the right direction, without giving away the full solution. People should not guess that you are an AI. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.
Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

prompt = new_test_df['prompt'][0]

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]
answer = pipe(messages, max_length=4096)[0]['generated_text'][-1]['content']
print(answer)

In [ ]:
from tqdm import tqdm

In [ ]:
# TEST inference

system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. Write directly where the mistake is. Do not write a long answer. Provide only a hint that guides the student in the right direction, without giving away the full solution. People should not guess that you are an AI. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.
Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

answers_test = []
for index, row in tqdm(new_test_df.iloc[:5].iterrows()):
  prompt = row['prompt']
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
  ]
  answer = pipe(messages, max_length=6048)[0]['generated_text'][-1]['content']
  answers_test.append(answer)

In [ ]:
len(answers_test)

In [ ]:
answer_test = pd.DataFrame(answers_test)

In [ ]:
answer_test.sample(2)

In [ ]:
for index, row in answer_test.iterrows():
    if '<Комментарий эксперта>:' in row[0]:
        row[0] = row[0][24:]

In [ ]:
#answer_test.to_csv('inference2time_without_training.csv')

In [ ]:
from typing import Callable

import pandas as pd
import torch
from transformers import BertModel, BertTokenizer

print("Loading models...", end="")
model_name = "DeepPavlov/rubert-base-cased-sentence"
emb_tokenizer = BertTokenizer.from_pretrained(model_name)
emb_model = BertModel.from_pretrained(model_name)
print("OK")


def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = emb_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = emb_model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding


def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])


def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])


def generate_submit(test_solutions: str, predict_func: Callable, save_path: str, use_tqdm: bool = True) -> None:
    bar = range(len(test_solutions))
    if use_tqdm:
        import tqdm

        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.id.iloc[i]
        solution_row = test_solutions.prompt.iloc[i]
        text = predict_func(solution_row)  # here you can do absolute whatever you want
        embedding = embedding2string(get_sentence_embedding(text))
        submit_df.loc[i] = [idx, text, embedding]
    submit_df.to_csv(save_path, index=False)

In [ ]:
def predict(prompt):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
    ]
    answer = pipe(messages, max_length=6048)[0]['generated_text'][-1]['content']
    #if '<Комментарий эксперта>:' in answer:
    #    answer = answer[24:] ########################
    return answer

In [ ]:
generate_submit(
        test_solutions=new_test_df,
        predict_func=predict,
        save_path="submit4_emae.csv",
        use_tqdm=True,
)

In [ ]:
df = pd.read_csv(path + "submit4_emae.csv")
df.head()